In [ ]:
!pip install --quiet ../input/pip-libraries/pyphen-0.11.0-py3-none-any.whl ../input/pip-libraries/textstat-0.7.1-py3-none-any.whl

# Infer

In [ ]:
model_list = ['distilroberta_base_regression_v3', 'distilroberta_base_regression_v6']

import pickle
import numpy as np

for model_name in model_list:
    !python ../input/commonlit-impl/src/1st_level/$model_name/infer.py $model_name

In [ ]:
import os
is_kaggle = 'KAGGLE_URL_BASE' in os.environ
if is_kaggle:
    pickle_path = '.'
else:
    pickle_path = '/content/kaggle_commonlit/pickle'

predicted_labels_ensembled = None
for model_name in model_list:
    with open(f'{pickle_path}/{model_name}.pkl', 'rb') as handle:
        predicted_labels = pickle.load(handle)
    if predicted_labels_ensembled is None:
        predicted_labels_ensembled = np.array(predicted_labels)
    else:
        predicted_labels_ensembled = predicted_labels_ensembled + np.array(predicted_labels)
predicted_labels_ensembled /= len(model_list)

In [ ]:
import pandas as pd
submission_df = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submission_df.target = predicted_labels_ensembled*1.0015
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)